In [1]:
from datetime import datetime
from tokenize import group

import polars as pl
import glob
from tqdm import tqdm

In [2]:
metadata = pl.read_csv('raw/metadata.csv', try_parse_dates=True)
metadata

user,start_date,end_date,length_days,length_years,potential_samples,actual_samples,missing_samples_abs,missing_samples_pct,contract_start_date,contract_end_date,contracted_tariff,self_consumption_type,p1,p2,p3,p4,p5,p6,province,municipality,zip_code,cnae
str,"datetime[μs, UTC]","datetime[μs, UTC]",f64,f64,i64,i64,i64,f64,date,date,str,str,f64,f64,f64,f64,f64,f64,str,str,i64,i64
"""00000c5a448d9faa097b761cc98036…",2022-05-30 01:00:00 UTC,2022-06-05 00:00:00 UTC,6.0,0.016427,144,144,0,0.0,2022-05-31,null,"""2.0TD""","""""",2.2,2.2,0.0,0.0,0.0,0.0,"""Gipuzkoa""",null,null,9329
"""0001b3b2f18c01c62ed9b2a87de7b4…",2017-05-31 01:00:00 UTC,2022-06-05 00:00:00 UTC,1831.0,5.013005,43944,43863,81,0.184326,2021-06-01,null,"""2.0TD""","""""",3.45,3.45,null,null,null,null,"""Bizkaia""",null,null,9820
"""0003de2700e20a1681d69fe287441d…",2017-05-31 01:00:00 UTC,2019-11-14 00:00:00 UTC,897.0,2.455852,21528,21476,52,0.241546,2019-09-02,2019-11-13,"""2.0A""","""""",4.6,null,null,null,null,null,"""Gipuzkoa""","""Donostia/San Sebastian""",20013,9820
"""0004150214d14a2b2e6f7075531e66…",2018-07-12 01:00:00 UTC,2022-06-05 00:00:00 UTC,1424.0,3.8987,34176,34169,7,0.020482,2021-06-01,null,"""2.0TD""","""""",9.2,9.2,null,null,null,null,"""Gipuzkoa""","""Irun""",20304,4759
"""000721f0fc6ccf02ae24b673939795…",2017-05-30 01:00:00 UTC,2022-06-04 00:00:00 UTC,1831.0,5.013005,43944,43815,129,0.293555,2021-06-01,null,"""2.0TD""","""""",4.4,4.4,null,null,null,null,"""Bizkaia""",null,null,9820
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""ffeddf4919e748beb50af7c6b73def…",2018-10-02 01:00:00 UTC,2021-11-03 00:00:00 UTC,1128.0,3.088296,27072,27069,3,0.011082,2021-06-01,null,"""2.0TD""","""""",6.0,6.0,null,null,null,null,"""Soria""",null,null,9820
"""ffefb9e27715b1c3fa5007c457de55…",2021-07-06 01:00:00 UTC,2022-06-05 00:00:00 UTC,334.0,0.914442,8016,8014,2,0.02495,2021-07-07,null,"""2.0TD""","""""",3.45,3.45,0.0,0.0,0.0,0.0,"""Bizkaia""","""Bilbao""",48012,9820
"""fff0adcd6cdb7434496a5222f03337…",2017-08-17 01:00:00 UTC,2022-06-05 00:00:00 UTC,1753.0,4.799452,42072,41102,970,2.305571,2021-06-01,null,"""2.0TD""","""""",4.6,4.6,null,null,null,null,"""Bizkaia""",null,null,9820


In [3]:
start_date_filter = pl.datetime(2017, 6, 1, time_zone='UTC')
end_date_filter = pl.datetime(2020, 3, 1, time_zone='UTC')


metadata_pre_house_1 = metadata.filter(
    pl.col("start_date") <= start_date_filter,
    pl.col("end_date") >= end_date_filter,
    (pl.col('cnae') > 9699),
    (pl.col('cnae') < 9849)
)
metadata_pre_house_1.head()

user,start_date,end_date,length_days,length_years,potential_samples,actual_samples,missing_samples_abs,missing_samples_pct,contract_start_date,contract_end_date,contracted_tariff,self_consumption_type,p1,p2,p3,p4,p5,p6,province,municipality,zip_code,cnae
str,"datetime[μs, UTC]","datetime[μs, UTC]",f64,f64,i64,i64,i64,f64,date,date,str,str,f64,f64,f64,f64,f64,f64,str,str,i64,i64
"""0001b3b2f18c01c62ed9b2a87de7b4…",2017-05-31 01:00:00 UTC,2022-06-05 00:00:00 UTC,1831.0,5.013005,43944,43863,81,0.184326,2021-06-01,null,"""2.0TD""","""""",3.45,3.45,null,null,null,null,"""Bizkaia""",null,null,9820
"""000721f0fc6ccf02ae24b673939795…",2017-05-30 01:00:00 UTC,2022-06-04 00:00:00 UTC,1831.0,5.013005,43944,43815,129,0.293555,2021-06-01,null,"""2.0TD""","""""",4.4,4.4,null,null,null,null,"""Bizkaia""",null,null,9820
"""001be27fc1f3e2daa58919a9de02e9…",2017-05-31 01:00:00 UTC,2022-06-05 00:00:00 UTC,1831.0,5.013005,43944,43887,57,0.129711,2021-06-01,null,"""2.0TD""","""""",3.3,3.3,null,null,null,null,"""Gipuzkoa""",null,null,9820
"""00264d835371489e86312529d5e5aa…",2017-05-31 01:00:00 UTC,2022-06-05 00:00:00 UTC,1831.0,5.013005,43944,41237,2707,6.160113,2021-06-01,null,"""2.0TD""","""""",3.45,3.45,null,null,null,null,"""Bizkaia""",null,null,9820
"""002da62dc54b7a789473182feddfad…",2017-05-31 01:00:00 UTC,2022-06-05 00:00:00 UTC,1831.0,5.013005,43944,43815,129,0.293555,2021-09-25,null,"""2.0TD""","""""",3.45,3.45,null,null,null,null,"""Navarra""","""Pamplona/Iruna""",31015,9820


In [4]:
metadata_pre_house_1_users = metadata_pre_house_1['user'].unique().to_list()

In [5]:
print(f"Post-COVID household users={len(metadata_pre_house_1_users)}")

Post-COVID household users=4928


# Combine CSV

In [6]:
source_folder = 'dataset/goi4_pre/imp_csv'
csv_files = glob.glob(f"{source_folder}/*.csv")

In [7]:
def combine(mask: list[str], csv_files: list[str], filename:str='dataset/goi4_pre/household_activities_2017.csv') -> pl.DataFrame:
    """
    Combine multiple CSV files into a single DataFrame, filtering by household profiles.
    """
    dataframes = pl.DataFrame()
    for csv_file in tqdm(csv_files):
        profile_id = csv_file.split('/')[-1].split('.')[0]
        if profile_id not in mask:
            continue

        df = pl.read_csv(csv_file, try_parse_dates=True, schema_overrides={'kWh':pl.Float64})
        df = df.with_columns(pl.lit(profile_id).alias('profile_id'))

        df = df.with_columns(
            pl.col("timestamp")
            .dt.replace_time_zone("UTC"))

        df = df.filter(pl.col('timestamp') >= start_date_filter,
                       pl.col('timestamp') <= end_date_filter)

        if not dataframes.columns:
            dataframes = df.clone()
            continue
        dataframes.extend(df)
    if filename:
        dataframes = dataframes.drop('imputed')
        dataframes.write_csv(filename)

    return dataframes

In [8]:
combine(metadata_pre_house_1_users, csv_files)

100%|██████████| 12149/12149 [00:35<00:00, 344.66it/s]


timestamp,kWh,profile_id
"datetime[μs, UTC]",f64,str
2017-06-01 00:00:00 UTC,0.11,"""3b71707954dc962ab68fe44fa46b62…"
2017-06-01 01:00:00 UTC,0.152,"""3b71707954dc962ab68fe44fa46b62…"
2017-06-01 02:00:00 UTC,0.023,"""3b71707954dc962ab68fe44fa46b62…"
2017-06-01 03:00:00 UTC,0.045,"""3b71707954dc962ab68fe44fa46b62…"
2017-06-01 04:00:00 UTC,0.066,"""3b71707954dc962ab68fe44fa46b62…"
…,…,…
2020-02-29 20:00:00 UTC,0.665,"""5793ddb25829ba3db566b882d7cf11…"
2020-02-29 21:00:00 UTC,1.448,"""5793ddb25829ba3db566b882d7cf11…"
2020-02-29 22:00:00 UTC,0.17,"""5793ddb25829ba3db566b882d7cf11…"


In [9]:
df_activities = pl.read_csv("dataset/goi4_pre/household_activities_2017.csv")
df_activities.head()

timestamp,kWh,profile_id
str,f64,str
"""2017-06-01T00:00:00.000000+000…",0.11,"""3b71707954dc962ab68fe44fa46b62…"
"""2017-06-01T01:00:00.000000+000…",0.152,"""3b71707954dc962ab68fe44fa46b62…"
"""2017-06-01T02:00:00.000000+000…",0.023,"""3b71707954dc962ab68fe44fa46b62…"
"""2017-06-01T03:00:00.000000+000…",0.045,"""3b71707954dc962ab68fe44fa46b62…"
"""2017-06-01T04:00:00.000000+000…",0.066,"""3b71707954dc962ab68fe44fa46b62…"


# Preprocess

## Interpolate

In [10]:
import pandas as pd

In [11]:
df = pd.read_csv("dataset/goi4_pre/household_activities_2017.csv")

In [12]:
df.head()

,timestamp,kWh,profile_id
0,2017-06-01T00:00:00.000000+0000,0.110,3b71707954dc962ab68fe44fa46b62d9a9b5348ce457bf...
1,2017-06-01T01:00:00.000000+0000,0.152,3b71707954dc962ab68fe44fa46b62d9a9b5348ce457bf...
2,2017-06-01T02:00:00.000000+0000,0.023,3b71707954dc962ab68fe44fa46b62d9a9b5348ce457bf...
3,2017-06-01T03:00:00.000000+0000,0.045,3b71707954dc962ab68fe44fa46b62d9a9b5348ce457bf...
4,2017-06-01T04:00:00.000000+0000,0.066,3b71707954dc962ab68fe44fa46b62d9a9b5348ce457bf...


In [13]:
df_ = df.copy()

In [14]:
df_['timestamp'] = pd.to_datetime(
    df_['timestamp'],
    format='%Y-%m-%dT%H:%M:%S.%f%z'
)

In [15]:
df_.head()

,timestamp,kWh,profile_id
0,2017-06-01 00:00:00+00:00,0.110,3b71707954dc962ab68fe44fa46b62d9a9b5348ce457bf...
1,2017-06-01 01:00:00+00:00,0.152,3b71707954dc962ab68fe44fa46b62d9a9b5348ce457bf...
2,2017-06-01 02:00:00+00:00,0.023,3b71707954dc962ab68fe44fa46b62d9a9b5348ce457bf...
3,2017-06-01 03:00:00+00:00,0.045,3b71707954dc962ab68fe44fa46b62d9a9b5348ce457bf...
4,2017-06-01 04:00:00+00:00,0.066,3b71707954dc962ab68fe44fa46b62d9a9b5348ce457bf...


In [16]:
df_ = df_.sort_values(by=['profile_id', 'timestamp'])

In [17]:
df_['timestamp'].min(), df_['timestamp'].max()

(Timestamp('2017-06-01 00:00:00+0000', tz='UTC'),
 Timestamp('2020-03-01 00:00:00+0000', tz='UTC'))

In [18]:
time_grid  = pd.date_range(df_['timestamp'].min(),
                           df_['timestamp'].max(),
                           freq='30min')
profile_ids = df_['profile_id'].unique()

In [19]:
idx = pd.MultiIndex.from_product(
    [profile_ids, time_grid],
    names=['profile_id','timestamp']
)

In [20]:
df_idx = df_.set_index(['profile_id','timestamp'])
df_30 = df_idx.reindex(idx)

In [21]:
df_30.head()

kWh
profile_id                                         timestamp                       
0001b3b2f18c01c62ed9b2a87de7b4e33e7836f786f7904... 2017-06-01 00:00:00+00:00  0.190
                                                   2017-06-01 00:30:00+00:00    NaN
                                                   2017-06-01 01:00:00+00:00  0.057
                                                   2017-06-01 01:30:00+00:00    NaN
                                                   2017-06-01 02:00:00+00:00  0.099

In [22]:
df_30['kWh_interpolated'] = (
    df_30
    .groupby('profile_id')['kWh']
    .transform(lambda s: s.interpolate(method='linear'))
)

In [23]:
df_30.head()

kWh  \
profile_id                                         timestamp                          
0001b3b2f18c01c62ed9b2a87de7b4e33e7836f786f7904... 2017-06-01 00:00:00+00:00  0.190   
                                                   2017-06-01 00:30:00+00:00    NaN   
                                                   2017-06-01 01:00:00+00:00  0.057   
                                                   2017-06-01 01:30:00+00:00    NaN   
                                                   2017-06-01 02:00:00+00:00  0.099   

                                                                              kWh_interpolated  
profile_id                                         timestamp                                    
0001b3b2f18c01c62ed9b2a87de7b4e33e7836f786f7904... 2017-06-01 00:00:00+00:00            0.1900  
                                                   2017-06-01 00:30:00+00:00            0.1235  
                                                   2017-06-01 01:00:00+00:00            0.0570  
                                                   2017-06-01 01:30:00+00:00            0.0780  
                                                   2017-06-01 02:00:00+00:00            0.0990

In [26]:
df_30 = df_30.drop(columns=['kWh'])
df_30 = df_30.rename(columns={'kWh_interpolated': 'kWh'})
df_30 = df_30.reset_index()

df_30['profile_id'] = [a[:7]for a in df_30['profile_id']]
df_30['timestamp'] = [a.replace('+00:00','.0000000') for a in df_30['timestamp']]

In [27]:
df_30.head()

kWh
profile_id                                         timestamp                        
0001b3b2f18c01c62ed9b2a87de7b4e33e7836f786f7904... 2017-06-01 00:00:00+00:00  0.1900
                                                   2017-06-01 00:30:00+00:00  0.1235
                                                   2017-06-01 01:00:00+00:00  0.0570
                                                   2017-06-01 01:30:00+00:00  0.0780
                                                   2017-06-01 02:00:00+00:00  0.0990

In [ ]:
df_30 = df_30.reset_index()

In [ ]:
df_17 = df_30[df_30['timestamp'] == '2017-06-02 00:00:00.0000000']
df_20 = df_30[df_30['timestamp'] == '2020-02-28 00:00:00.0000000']

In [ ]:
df_17 = df_17[df_17['kWh']==0.0]
df_20 = df_20[df_20['kWh']==0.0]

In [ ]:
users = list(df_17['profile_id'].unique())
users.extend(list(df_20['profile_id'].unique()))
users = list(set(users))

len(users)

In [ ]:
df_4676 = df_30.drop(df_30.index[df_30['profile_id'].isin(users)])

In [10]:
df_4676.to_csv("dataset/goi4_pre/GoiEner_pre.csv")

In [1]:
import pandas as pd
df_30 = pd.read_csv("dataset/goi4_pre/GoiEner_pre.csv")

In [2]:
df_30.head()

,profile_id,timestamp,kWh
0,0001b3b,2017-06-01 00:00:00.0000000,0.190
1,0001b3b,2017-06-01 00:30:00.0000000,0.124
2,0001b3b,2017-06-01 01:00:00.0000000,0.057
3,0001b3b,2017-06-01 01:30:00.0000000,0.078
4,0001b3b,2017-06-01 02:00:00.0000000,0.099
